In [9]:
import pandas as pd
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [10]:
news = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [11]:
news = news.dropna(subset=['content', 'source'])
news[:5]

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [12]:
news = news.sample(5000)

In [13]:
news['xinhua'] = np.where(news['source'].str.contains('新华'), 1, 0)
news[:5]

,id,author,source,content,feature,title,url,xinhua
51233,38379,周咏缗,新华社,新华社图表，北京，2017年5月5日\n图表：索马里霍乱疫情已造成618人死亡\n新华社记者...,"{""type"":""其它"",""site"":""新华社"",""url"":""http://home.x...",（图表）[国际]索马里霍乱疫情已造成618人死亡,http://home.xinhua-news.com/gdsdetailxhsnew/20...,1
59711,29901,王永卓,新华社,新华社图表，北京，2017年5月12日\n图表：我国启动首个“文化和自然遗产日”\n新华社记...,"{""type"":""文化娱乐"",""site"":""新华社"",""url"":""http://home...",（图表）[文化和自然遗产日]我国启动首个“文化和自然遗产日”,http://home.xinhua-news.com/gdsdetailxhsnew/20...,1
72679,16933,李尕,新华社,新华社照片，益阳（湖南），2017年5月24日\n校园T台上演最美校服秀\n5月23日，模特...,"{""type"":""文化娱乐"",""site"":""新华社"",""url"":""http://home...",（文化）（2）校园T台上演最美校服秀,http://home.xinhua-news.com/gdsdetailxhsnew/21...,1
14767,74845,JOHN MACDOUGALL,新华社,新华社照片，外代，2017年3月31日\n（外代二线）花样滑冰——世锦赛双人滑：隋文静/韩聪...,"{""type"":""体育"",""site"":""新华社"",""url"":""http://home.x...",[12]（外代二线）花样滑冰——世锦赛双人滑：隋文静/韩聪夺冠,http://home.xinhua-news.com/gdsdetailxhsnew/17...,1
69635,19977,张曼,新华社,新华社法国戛纳５月２０日电专访：中国艺术电影生存环境日益好转——访中国青年导演李睿珺\n...,"{""type"":""文化娱乐"",""site"":""新华社"",""url"":""http://home...",（文化）专访：中国艺术电影生存环境日益好转——访中国青年导演李睿珺,http://home.xinhua-news.com/gdsdetailxhsnew/21...,1


In [31]:
y = news['xinhua'].tolist()
y = np.array(y)

In [14]:
np.where(news['source'].str.contains('新华'), 1, 0)

array([1, 1, 1, ..., 1, 1, 1])

In [81]:
len(news[news['xinhua']==0])

478

In [16]:
sample = news['content'].tolist()

In [17]:
a = ''.join(re.findall(re.compile('\w|\d'), sample[0]))
b = ' '.join(jieba.cut(a))
b

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/fv/1ndmyk4d1017_0c5l6w781240000gp/T/jieba.cache
Loading model cost 0.978 seconds.
Prefix dict has been built succesfully.


'新华社 图表 北京 2017 年 5 月 5 日 n 图表 索马里 霍乱 疫情 已 造成 618 人 死亡 n 新华社 记者 周咏 缗 编制 n'

In [18]:
def cut(content):
    def remove_extra_char(content):
        content =  [''.join(re.findall(re.compile('\w|\d'), con)) for con in content]
        return content
    
    content = remove_extra_char(content)
    
    jieba.enable_parallel(4)
    
    return [' '.join(jieba.cut(con)) for con in content]

In [19]:
tokens = cut(sample)
tokens[:5]

['新华社 图表 北京 2017 年 5 月 5 日 n 图表 索马里 霍乱 疫情 已 造成 618 人 死亡 n 新华社 记者 周咏 缗 编制 n',
 '新华社 图表 北京 2017 年 5 月 12 日 n 图表 我国 启动 首个 文化 和 自然遗产 日 n 新华社 记者 王永卓 编制',
 '新华社 照片 益阳 湖南 2017 年 5 月 24 日 n 校园 T台 上演 最美 校服 秀 n5 月 23 日 模特 在 展示 参赛 校服 设计 作品 n5 月 23 日 第二届 旭荣杯 最美 校服 设计 大赛 在 湖南 工艺美术 职业 学院 举行 由 学生 组成 的 模特 表演队 在 校园 T台 上 展示 19 个 系列 的 114 款 校服 这些 校服 大多 由 在校 师生 与 服装 企业 合作 设计 制作 是 校企 合作 职业 教育 模式 的 一次 成果 展示 n 新华社 记者 李尕摄',
 '新华社 照片 外代 2017 年 3 月 31 日 n 外代 二线 花样滑冰 世锦赛 双人滑 隋 文静 韩聪 夺冠 n3 月 30 日 中国 选手 隋 文静 左 韩聪 在 颁奖仪式 上 庆祝 n 当日 在 芬兰 赫尔辛基 举行 的 2017 年 花样滑冰 世锦赛 双人滑 比赛 决出 冠亚 季军 中国 选手 隋 文静 韩聪以 23206 分 的 总成绩 夺得 冠军 n 新华社 法新',
 '新华社 法国 戛纳 ５ 月 ２ ０ 日电 专访 中国 艺术 电影 生存环境 日益 好转 访 中国 青年 导演 李睿珺 n 新华社 记者 张曼 姜楠 n 中国 艺术 电影 生存环境 日益 好转 融资 和 发行 渠道 更加 畅通 给 了 年轻 导演 更 多 机会 中国 青年 导演 李睿珺 近日 在 第 ７ ０ 届 戛纳 电影节 期间 接受 新华社 记者 专访 时说 他 的 新作 路过 未来 入围 本届 戛纳 电影节 一种 关注 单元 n 李睿珺 从 ２ ０ ０ ６ 年 开始 导演 生涯 已经 数次 入围 重要 国际 电影节 ２ ０ １ ２ 年 其 作品 告诉 他们 我 乘 白鹤 去 了 入选 威尼斯 电影节 地平线 单元 ２ ０ １ ４ 年 其 作品 家 在 水草 丰茂 的 地方 入围 东京国际 电影节 主 竞赛 单元 路过 未来 是 他 的 第五部 作品 也 是 唯一 入围 本届 戛纳 

In [20]:
with open('baidu_stop_word.txt', 'r') as f:
    baidu_stop_words = f.read()
baidu_stop_words = baidu_stop_words.split()

In [21]:
vectorizer = TfidfVectorizer(stop_words = baidu_stop_words)
X = vectorizer.fit_transform(tokens)
X.shape

/Users/lin/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(5000, 68987)

In [21]:
#vectorizer.vocabulary_

In [22]:
tf_idf_vector = X.toarray()

In [23]:
tf_idf_vector.shape

(5000, 68987)

In [24]:
vectorizer.vocabulary_["这是"]

61935

In [25]:
np.where(tf_idf_vector[0])

(array([  777,  1760, 16012, 20005, 21593, 36585, 41921, 48477, 52392,
        53445, 58576, 66165]),)

In [26]:
pca =  PCA(n_components = 100)
#pca_train_x = tf_idf_vector - np.mean(tf_idf_vector)
pca_train_x = pca.fit_transform(tf_idf_vector)
pca_train_x.shape

(5000, 100)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(pca_train_x, y, test_size=0.33, random_state=42)

In [33]:
def performance(y, y_hat):
    print("precision: {}".format(precision_score(y, y_hat)))
    print("recall: {}".format(recall_score(y, y_hat)))
    print("roc_auc: {}".format(roc_auc_score(y, y_hat)))

In [34]:
class KNNCluster:
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y
        
    def run(self, x):
        x = x.reshape(1, -1)
        idx = np.argmax(cosine_similarity(self.train_x, x))
        return self.train_y[idx]
    
    def predict(self, test):
        y_hat = []
        for x in test:
            y = self.run(x)
            y_hat.append(y)
        
        return np.array(y_hat)
    
    def get_cos(self, v1, v2):
        return cosine(v1, v2)

In [35]:
knn = KNNCluster(X_train, y_train)

In [36]:
knn_y_hat = knn.predict(X_test)

In [37]:
performance(y_test, knn_y_hat)

precision: 0.9555256064690026
recall: 0.9447035309793471
roc_auc: 0.7508752554225594


In [38]:
from sklearn.neighbors import KNeighborsClassifier

In [39]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [40]:
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [41]:
sk_knn_y_hat = neigh.predict(X_test)

In [42]:
performance(y_test, sk_knn_y_hat)

precision: 0.9598366235534377
recall: 0.9393737508327782
roc_auc: 0.771700298235181


In [43]:
full_X_train, full_X_test, full_y_train, full_y_test = train_test_split(tf_idf_vector, y, test_size=0.33, random_state=42)

## SVM

In [44]:
from sklearn.svm import SVC

In [45]:
clf = SVC(kernel = 'rbf')

In [46]:
clf.fit(X_train, y_train)

/Users/lin/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [47]:
svm_y_hat = clf.predict(X_test)

In [48]:
performance(y_test, svm_y_hat)

precision: 0.9096969696969697
recall: 1.0
roc_auc: 0.5


## Bayes

In [49]:
from sklearn.naive_bayes import GaussianNB

In [50]:
clf = GaussianNB()

In [51]:
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [82]:
bayes_y_hat = clf.predict(X_test)

In [83]:
performance(y_test, bayes_y_hat)

precision: 0.974
recall: 0.6489007328447701
roc_auc: 0.737202044274734


In [105]:
from scipy.stats import norm
from sklearn import preprocessing

In [106]:
classifier = set(y_train)

In [110]:
mean = [np.mean(X_train[y_train==cls, :], axis=0) for cls in classifier]
mean

[array([-7.99157530e-02, -1.05005705e-02, -1.25519140e-02, -2.41024762e-02,
        -1.43547107e-02, -1.49137082e-02,  1.57758448e-02, -1.97748267e-02,
        -2.11251492e-02,  4.68444938e-04, -1.28199192e-02, -1.34941658e-02,
        -4.68386512e-04,  1.26226493e-02, -6.62611432e-03, -1.13250678e-03,
        -3.22201872e-03, -9.67523920e-03,  1.38022116e-02,  2.32744543e-03,
         3.43821410e-03, -3.55811929e-04, -3.10601043e-03, -1.97105065e-03,
        -5.78621369e-04,  2.13909254e-03, -1.13873110e-03,  4.50713030e-03,
         7.55622555e-04,  1.66086117e-03,  8.32591643e-03,  1.80330119e-03,
        -2.80741227e-03,  2.28528851e-03,  3.98521187e-03, -5.76342827e-03,
        -6.67700723e-03, -5.97966543e-03,  3.02082513e-03,  6.47509477e-04,
        -3.31091714e-03,  7.75537074e-03,  7.76790967e-03, -2.67579298e-03,
        -6.71288676e-03,  3.76017029e-03,  4.27594061e-03, -5.14653241e-04,
        -9.16720674e-05,  3.52922949e-03,  4.89449679e-03,  3.51400188e-03,
        -5.9

In [165]:
class BayesClassifier:
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y
        
        self.prepare()
        
    def prepare(self):
        self.classifier = list(set(self.train_y))
        
        self.mean = [np.mean(self.train_x[self.train_y==cls, :], axis=0) for cls in self.classifier]
        self.std = [np.std(self.train_x[self.train_y==cls, :], axis=0) for cls in self.classifier]
        self.mean = np.array(self.mean)
        self.std = np.array(self.std)
    
        self.p_classifier = [len(self.train_x[self.train_y==cls]) / len(self.train_x) for cls in self.classifier]
        
    def likehood(self, x, cls_idx):
        mean = self.mean[cls_idx,:]
        std = self.std[cls_idx,:]
        p_cls = self.p_classifier[cls_idx]
        
        joint = -np.sum(np.log(np.sqrt(2 * np.pi) * std)) - \
                    np.sum(np.power(x - mean, 2) / (2 * np.power(std, 2)))

        return joint + np.log(p_cls)
    
    def predict(self, test):
        y_hat = []
        
        for vector in test: 
            p_y = [self.likehood(vector, i) for i in range(len(self.classifier))]
        
            y_hat.append(self.classifier[np.argmax(p_y)])
                
        return np.array(y_hat)

In [166]:
bayes = BayesClassifier(X_train, y_train)

In [167]:
bayes_y_hat = bayes.predict(X_test)
sum(bayes_y_hat)

1000

In [168]:
performance(y_test, bayes_y_hat)

precision: 0.974
recall: 0.6489007328447701
roc_auc: 0.737202044274734
